# Pointcloud Segmentation with RANSAC and DBSCAN

In this notebook we will segment the images using pointcloud processing algorithms. First import libraries

In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import json
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load Data and Create Point Cloud

Load sample images and create pointcloud object
1. Load camera parameters
2. Load depth and color images
3. Create RGBD image object
4. Create Intrinsics object
5. Create PointCloud object

In [3]:
# variables to easily change sample image
obj = 'pimenton'
view = 'front'
num = 0

# load params
with open(os.path.join('img',obj,f'calib_data_{view}.json')) as json_file:
    cam_params = json.load(json_file)

# load images
color = o3d.io.read_image(os.path.join('img', obj, f'{view}_color_{num}.jpg'))
depth = o3d.io.read_image(os.path.join('img', obj, f'{view}_depth_{num}.png'))

# create RGBD image
rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, depth, convert_rgb_to_intensity=False)

# create intrinsics object
intrinsics = o3d.camera.PinholeCameraIntrinsic(
    cam_params['depth intrinsics']['width'],
    cam_params['depth intrinsics']['height'],
    cam_params['depth intrinsics']['fx'],
    cam_params['depth intrinsics']['fy'],
    cam_params['depth intrinsics']['ppx'],
    cam_params['depth intrinsics']['ppy']
)

# create Point Cloud
pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd,
    intrinsics,
    project_valid_depth_only=True
)
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

PointCloud with 271008 points.

Visualize point cloud (opens in another window)

In [4]:
o3d.visualization.draw_geometries([pcd])

## Plane Detection with RANSAC

### One plane

In [4]:
plane_model, inliers = pcd.segment_plane(
    distance_threshold=0.02,
    ransac_n=3,
    num_iterations=1000
)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.03x + -0.02y + 1.00z + 0.75 = 0


3D Visualization (Opens in another window)

In [5]:
inlier_pcd = pcd.select_by_index(inliers)
inlier_pcd.paint_uniform_color([1.0, 0.0, 0.0])
outlier_pcd = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd])

This will Detect the plane corresponding to the wall behind the object. If we map all this points to $[0,0,0]$ we are effectively removing the plane 

In [6]:
# map plane inliers to origin
points = np.asarray(pcd.points).copy()
points[inliers] = [0, 0, 0]
# create a pointcloud from the set of mapped points
pcd_proc = o3d.geometry.PointCloud(pcd)
pcd_proc.points = o3d.utility.Vector3dVector(points)

In [7]:
o3d.visualization.draw_geometries([pcd_proc])
#o3d.visualization.draw_geometries([pcd])

### Several planes in a Loop

Removing inliers in each iteration for visualization

In [8]:
segment_models = []
segments = []
max_plane_idx = 4
rest = o3d.geometry.PointCloud(pcd)

for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_model, inliers = rest.segment_plane(
        distance_threshold=0.009,
        ransac_n=3,
        num_iterations=1000
    )

    # for visualization
    segment_models.append(segment_model)
    segments.append(rest.select_by_index(inliers))
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print(f"Pass {i+1}/{max_plane_idx} done")

Pass 1/4 done
Pass 2/4 done
Pass 3/4 done
Pass 4/4 done


The equations for the planes found are:

In [ ]:
for a,b,c,d in segment_models:
    print(f"{a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

In [9]:
o3d.visualization.draw_geometries(segments)

In [10]:
o3d.visualization.draw_geometries([rest])

In [11]:
o3d.visualization.draw_geometries(segments + [rest])

Without removing points in the loop. Mapping to zero the planes detected leads to eventual false planes detected. Mapping them to `nan` partialy solves this problem but we can still have some false positive planes.

In [12]:
max_plane_idx = 4
pcd_proc = o3d.geometry.PointCloud(pcd)
vis = False
for i in range(max_plane_idx):
    plane_model, inliers = pcd_proc.segment_plane(
        distance_threshold=0.009,
        ransac_n=3,
        num_iterations=1000
    )
    if vis:
        inlier_pcd = pcd_proc.select_by_index(inliers)
        outlier_pcd = pcd_proc.select_by_index(inliers, invert=True)
        inlier_pcd.paint_uniform_color([1,0,0])
        o3d.visualization.draw_geometries([inlier_pcd, outlier_pcd])
    points = np.array(pcd_proc.points)
    points[inliers] = [np.nan]*3
    pcd_proc.points = o3d.utility.Vector3dVector(points)

In [13]:
o3d.visualization.draw_geometries([pcd_proc])

### RANSAC refinement with DBSCAN

Refining RANSAC with DBSCAN will allow us to filter false positives

In [14]:
segment_models = []
segments = []
max_plane_idx = 4
pcd_proc = o3d.geometry.PointCloud(pcd)
d_thresh = 0.009

for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    plane_model, inliers = pcd_proc.segment_plane(
        distance_threshold=d_thresh,
        ransac_n=3,
        num_iterations=1000
    )
    segment_models.append(plane_model)
    segments.append(pcd_proc.select_by_index(inliers))
    # dbscan refinement
    labels = np.array(segments[i].cluster_dbscan(
        eps=10*d_thresh,
        min_points=10
    ))
    unique_labels = np.unique(labels)
    # numbe of points of each cluster
    candidates = [len(np.where(labels==j)[0]) for j in unique_labels]
    # best candidate based on largest cluster
    best_plane = int(unique_labels[np.where(candidates==np.max(candidates))[0]])    
    print(f"Best plane is {best_plane} out of {len(unique_labels)} candidates")
    pcd_proc = pcd_proc.select_by_index(inliers, invert=True) + \
        segments[i].select_by_index(list(np.where(labels!=best_plane)[0]))
    segments[i] = segments[i].select_by_index(list(np.where(labels==best_plane)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print(f"Pass {i+1}/{max_plane_idx} done")

Best plane is 0 out of 1 candidates
Pass 1/4 done
Best plane is 0 out of 1 candidates
Pass 2/4 done
Best plane is 0 out of 1 candidates
Pass 3/4 done
Best plane is 1 out of 3 candidates
Pass 4/4 done


In [15]:
o3d.visualization.draw_geometries(segments)

In [16]:
o3d.visualization.draw_geometries([pcd_proc])

In [17]:
o3d.visualization.draw_geometries(segments+[pcd_proc])

## Clustering with DBSCAN

With the remaining points, we perform DBSCAN clustering to find our object of interest

In [18]:
labels = np.array(
    rest.cluster_dbscan(eps=0.02, min_points=10)
)
max_label = labels.max()
print(f"Point clod has {max_label} clusters")
colors = plt.get_cmap("tab10")(labels / max_label if max_label > 0 else 1)
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

Point clod has 18 clusters


In [30]:
o3d.visualization.draw_geometries(segments+[rest])

In [29]:
o3d.visualization.draw_geometries([rest])

In [26]:
# find largest cluster
cluster_sizes = [len(np.where(labels==j)[0]) for j in np.unique(labels)]
biggest_cluster=int(np.unique(labels)[np.where(cluster_sizes==np.max(cluster_sizes))[0]])
biggest_cluster

7

In [27]:
o3d.visualization.draw_geometries(
    [rest.select_by_index(np.where(labels==biggest_cluster)[0])]
)

In [28]:
o3d.visualization.draw_geometries(
    [pcd]+[rest.select_by_index(np.where(labels==biggest_cluster)[0])]
)